# Chapter 6: Design Patterns with First-Class Functions

From [Wikipedia](https://en.wikipedia.org/wiki/Software_design_pattern):

> In software engineering, a **software design pattern** is a general, reusable solution to a commonly occurring problem within a given context in software design. It is not a finished design that can be transformed directly into source or machine code. Rather, it is a description or template for how to solve a problem that can be used in many different situations. Design patterns are formalized best practices that the programmer can use to solve common problems when designing an application or system.

In the context of languages with first-class functions, the [Strategy](https://en.wikipedia.org/wiki/Strategy_pattern), [Command](https://en.wikipedia.org/wiki/Command_pattern), [Template Method](https://en.wikipedia.org/wiki/Template_method_pattern), and [Visitor](https://en.wikipedia.org/wiki/Visitor_pattern) patterns can be reimagined.

The general idea is that one can replace instances of some participant in these patterns with simple functions, thus reducing boilerplate code.

## Case Study: Refactoring Strategy

The classic [Strategy](https://en.wikipedia.org/wiki/Strategy_pattern) design pattern is summarised like this in [*Design Patterns*](https://www.bookdepository.com/Design-Patterns/9780201633610):

> Define a family of algorithms, encapsulate each one, and make the interchangeable. Strategy lets the algorithm vary independently from clients using it.

**Example 6-1.** An online store has the following discount rules:

* Customers with 1000 or more fidelity points get a global
l 5% discount per order.

* A 10% discount is applied to each line item with 20 or more units in the same order.

* Order with at least 10 distinct items get a 7% global discount.

The participants in the Strategy design pattern are:

* *Context*: Provides a service by delegating some computation to interchangeable components that implement alternative algorithms.

* *Strategy*: The interface common to the components that implement the different algorithms.

* *Concrete Strategy*: One of the concrete subclasses of Strategy.

In this example, before instantiating an order, the system would somehow select a promotional discount strategy and pass it to the `Order` constructor. How to do this is beyond the scope of the example.

In [4]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")

@dataclass
class LineItem:
    product: str
    quantity: int
    price: float
        
    def total(self):
        return self.price * self.quantity
    
    
# Context
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.cart())
        return self.__total() # Note double leading underscore to prevent accidental modification
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() * self.discount()
    
    def __repr__(self):
        fmt = "<Order total: {:.2f} due {.:2f}"
        return fmt.format(self.total(), self.due())

    
# Strategy
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """Return discount as positive dollar amount."""
      
    
# Concrete strategies
class FidelityPromo(Promotion):
    
    def discount(self, order):
        """5% discount for customers with 1000 or more fidelity points."""
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

    
class BulkItemPromo(Promotion):
    
    def discount(self, order):
        """10% discount for line orders with 20 or more units."""
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
    
class LargeOrderPromo(Promotion):
    
    def discount(self, order):
        "7% discount on orders with at least 10 distinct items."
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10 :
            return order.total() * .07
        else:
            return 0    
